<a href="https://colab.research.google.com/github/alfredkj7/Image-Classifiers/blob/master/Multiclass_Image_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mounting the google drive which contains the dataset
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
#Change the corresponding directories in the following
base_dir = '/content/gdrive/My Drive/Datasets/[Dataset directory]'
train_dir = os.path.join(base_dir, '[training set directory]')
validation_dir = os.path.join(base_dir, '[validation set directory]')

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#Downloading weights of a pretrained model
!wget https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape = (300, 300, 3), 
                                include_top = False, 
                                weights = None)
pre_trained_model.load_weights(local_weights_file)
for layer in pre_trained_model.layers:
    layer.trainable = False
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(3, activation='softmax')(x)
model = Model(pre_trained_model.input, x)

In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Callback to prevent further epochs
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('loss') < 0.05 and logs.get('val_loss') < 0.05:
            self.model.stop_training = True


callbacks = myCallback()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")

validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,  
    target_size=(300, 300),  
    batch_size=254,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,  
    target_size=(300, 300),  
    batch_size=130,
    class_mode='categorical')

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=10, #Note: no. of images in the training set = steps_per_epoch x batch_size
    epochs=30,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=3, #Note: no. of images in the validation set = validation_steps x batch_size
    callbacks=[callbacks])

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image

#Upload new image
uploaded=files.upload()
for fn in uploaded.keys():
  path='/content/' + fn
  img=image.load_img(path, target_size=(300, 300))
  x=image.img_to_array(img)
  x=x/255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn + " is in category", classes[0].index(max(classes[0])) + 1)

In [ ]:
drive.flush_and_unmount()
import os, signal
os.kill(os.getpid(), signal.SIGKILL)